In [12]:
import folium
import pandas as pd

# Import data
%store -r data_sampah
%store -r koordinat_BDG

# Mengubah kolom tanggal menjadi tipe data datetime
data_sampah['tanggal'] = pd.to_datetime(data_sampah['tanggal'], format='%m/%d/%Y')

# Membuat kolom baru untuk mengelompokkan kolom tanggal berdasarkan tahun
data_sampah['tahun'] = data_sampah['tanggal'].dt.to_period('Y')

# Mengelompokkan berdasarkan tahun dan kecamatan, kemudian menghitung jumlah sampah
grouped_data = data_sampah.groupby(['tahun', 'kecamatan']).agg({
    'jumlah_sampah': 'sum'
}).reset_index()

# Menggabungkan dengan data koordinat dan data sampah
merged_data = pd.merge(grouped_data, koordinat_BDG, on='kecamatan')

koordinat_BDG['Lintang Selatan'] = koordinat_BDG['Lintang Selatan'].astype(str)
koordinat_BDG['Bujur Timur'] = koordinat_BDG['Bujur Timur'].astype(str)

# Membersihkan dan mengonversi kolom 'Lintang Selatan' dan 'Bujur Timur' ke tipe data numerik
koordinat_BDG['Lintang Selatan'] = pd.to_numeric(koordinat_BDG['Lintang Selatan'].str.replace(';', '').str.strip(), errors='coerce')
koordinat_BDG['Bujur Timur'] = pd.to_numeric(koordinat_BDG['Bujur Timur'].str.replace(';', '').str.strip(), errors='coerce')

# Memasukan longitude dan latitud(rata rata supaya berada ditengah)
bandung_map = folium.Map(location=[koordinat_BDG['Lintang Selatan'].mean(), koordinat_BDG['Bujur Timur'].mean()], zoom_start=13)
for index, row in merged_data.iterrows():
    # Menentukan warna marker berdasarkan jumlah sampah
    if row['jumlah_sampah'] > 1500:
        color = 'red'
    elif row['jumlah_sampah'] > 1000:
        color = 'orange'
    else:
        color = 'green'

    # Membuat HTML untuk Tooltip
    tooltip_html = f"""
    <h4>{row['kecamatan']}</h4>
    <h5>Tahun: {row['tahun']}</h5>
    <h5>Jumlah Sampah: {row['jumlah_sampah']} Ton</h5>
    """

    # Menambahkan marker untuk setiap baris dengan warna berdasarkan jumlah sampah
    marker = folium.Marker(
        location=[row['Lintang Selatan'], row['Bujur Timur']],
        tooltip=tooltip_html,
        icon=folium.Icon(color=color),
    )
    marker.add_to(bandung_map)


bandung_geojson = bandung.to_json()
folium.GeoJson(bandung_geojson).add_to(bandung_map)
# Menampilkan peta di notebook
display(bandung_map)

%store bandung_map


ValueError: Location should consist of two numerical values, but ' 107.660193;' of type <class 'str'> is not convertible to float.

In [ ]:
import folium
import pandas as pd
import geopandas as gpd
filename = 'kecBandung.json'
bandung = gpd.read_file(filename)
# Import file
data_BDG = pd.read_csv('../../datasets/Dataset_dummy_v2.csv')
koordinat_BDG = pd.read_csv('../../Koordinat_kecamatan_kota_bandung.csv')

# Mengubah kolom tanggal menjadi tipe data datetime
data_BDG['tanggal'] = pd.to_datetime(data_BDG['tanggal'], format='%m/%d/%Y')

# Membuat kolom baru untuk mengelompokkan kolom tanggal berdasarkan bulan
data_BDG['tahun'] = data_BDG['tanggal'].dt.to_period('Y')

# Mengelompokkan berdasarkan bulan dan kecamatan, kemudian menghitung jumlah sampah
grup_tahun_kecamatan = data_BDG.groupby(['tahun', 'kecamatan']).agg({
    'jumlah_sampah': 'sum'
}).reset_index()

# Menggabungkan dengan data koordinat dan data sampah
data_gabung = pd.merge(grup_tahun_kecamatan, koordinat_BDG, on='kecamatan')

# Memasukan longitude dan latitud(rata rata supaya berada ditengah)
bandung_map = folium.Map(location=[koordinat_BDG['Lintang Selatan'].mean(), koordinat_BDG['Bujur Timur'].mean()], zoom_start=13)
# Perulangan untuk mengecek setiap kolom
for index, row in data_gabung.iterrows():
    # Menentukan warna marker berdasarkan jumlah sampah
    if row['jumlah_sampah'] > 1500:
        color = 'red'
    elif row['jumlah_sampah'] > 1000:
        color = 'orange'
    else:
        color = 'green'

    # Menambahkan marker untuk setiap baris dengan warna berdasarkan jumlah sampah
    popup_text = f"Jumlah Sampah Kecamatan {row['kecamatan']} {row['jumlah_sampah']} Ton per tahun "
    marker = folium.Marker(location=[row['Lintang Selatan'], row['Bujur Timur']], popup=popup_text, icon=folium.Icon(color=color))
    marker.add_to(bandung_map)

bandung_geojson = bandung.to_json()
folium.GeoJson(bandung_geojson).add_to(bandung_map)

# Menampilkan peta di notebook
display(bandung_map)